<a href="https://colab.research.google.com/github/gherbin/ComputerVisionKUL/blob/master/CV_Group9_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all packages

In [0]:
import os
import cv2
import tarfile
import zipfile
import shutil

import numpy as np
import random
import logging

from urllib import request
from socket import timeout
from urllib.error import HTTPError, URLError

from google.colab import drive
from google.colab.patches import cv2_imshow
from distutils.dir_util import copy_tree

# added for HOG
from skimage.feature import hog
from skimage import exposure
from matplotlib import pyplot as plt

logging.basicConfig(level=logging.DEBUG)

mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)


Several utils functions


In [0]:
def pretty_return_dict_size(my_dict):
    output_list = ["\n"]
    for k in my_dict.keys():
        output_list.append(str(k))
        output_list.append(":")
        output_list.append(str(len(my_dict[k])))
        output_list.append("\n")
    return ''.join(output_list)

def show_images_from_dict(my_dict, show_index = False):
    for k in my_dict.keys():
        logging.debug("@------------------- Images of " + str(k) + " -------------------@")
        index = 0
        for img in my_dict[k]:
            if show_index:
                logging.debug("Image index: " + index)
                index+=1
            cv2_imshow(img)



In [0]:
base_path = "/content/sample_data/CV__Group_assignment"

if not os.path.isdir(base_path):
  os.makedirs(base_path)

vgg_face_dataset_url = "http://www.robots.ox.ac.uk/~vgg/data/vgg_face/vgg_face_dataset.tar.gz"

with request.urlopen(vgg_face_dataset_url) as r, open(os.path.join(base_path, "vgg_face_dataset.tar.gz"), 'wb') as f:
  f.write(r.read())

with tarfile.open(os.path.join(base_path, "vgg_face_dataset.tar.gz")) as f:
  f.extractall(os.path.join(base_path))

trained_haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"

with request.urlopen(trained_haarcascade_url) as r, open(os.path.join(base_path, "haarcascade_frontalface_default.xml"), 'wb') as f:
    f.write(r.read())


# New Section

How to generate datasets
A = Emma Stone

1.   A --> Emma Stone
2.   B --> Bradley Cooper
3.   C --> Jane Levy
4.   D --> Marc Blucas


#Idea to get the images dataset
## For A and B
1. define a seed for A, and a seed for B
2. generate a number based on this seed
3. using this number, select 50 images from the list of 1000 images provided in the database
4. Select 30 out of the 50 images obtained. This constitutes the original dataset (Training and Test) for A and B.

## For C and D
1. define a seed for C, and a seed for D
2. generator a number based on this seed
3. using this number, select 20 images from the lsit of 1000 images provided in the database
4. Select 10 out of the 20 images obtained. This constutes the original dataset for C and D.


Start from clean sheet


In [0]:
path_datasets = r"/content/datasets/"
path_discard = r"/content/discard/"
path_database = r"/content/DATABASE/"

try:
    shutil.rmtree(path_database)
    shutil.rmtree(path_datasets)
    shutil.rmtree(path_discard)
except:
    pass

Create required folders

In [0]:
file_info = path_database+ r"info_retrieved.txt"

try: 
    os.mkdir(path_database)
    os.mkdir(path_datasets) 
    os.mkdir(path_discard)
except OSError as error: 
    logging.error(error) 



Instead of randomly download from web, take images from "clean" and controled repository containing the 180 images downloaded once and for all. 

For that purpose, we use a github public repository created specifically for that matter. The address of this public repository is hardcoded.

In [0]:
# drive.mount('/content/drive')
load_from_local_drive = True
if load_from_local_drive:
    
    !wget https://raw.githubusercontent.com/gherbin/cv_group9_database_replica/master/DATABASE-20200318T142918Z-001.zip

    with zipfile.ZipFile("DATABASE-20200318T142918Z-001.zip", 'r') as zip_ref:
        zip_ref.extractall()
    !rm -r "DATABASE-20200318T142918Z-001.zip"

path_, dirs_, files = next(os.walk(path_database))
if len(files) == 180+1:
    logging.info("Successful database retrieval")
elif load_from_local_drive:
    logging.error("Most Likely problem with database retrieval, number of files = " + str(len(files)))
else:
    logging.info("No database images retrieved yet")

XXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXX

In [0]:
personA = "Emma_Stone.txt"
personC = "Jane_Levy.txt"
personB = "Bradley_Cooper.txt"
personD = "Marc_Blucas.txt"
persons = [personA, personB, personC, personD]
datasets_dict = {}
images_size = {}
images_size[personA] = 60
images_size[personB] = 60
images_size[personC] = 30
images_size[personD] = 30

total_images_size = sum(images_size.values())

# Dictionary containing the ids of the pictures downloaded from internet
vgg_ids = {}
for p in persons:
    vgg_ids[p] = []


Populating DB - Run only if database folder is empty

In [0]:
confirmation = False
if confirmation:
    try:
        shutil.rmtree(path_database)
    except:
        pass 
    try:
        os.mkdir(path_database)
    except:
        pass 

    fo = open(file_info, "w+")

    # images = {}
    # images_nominal_indices = {}
    for person in persons:
        logging.debug("Taking care of: " + str(person))
        random.seed(person)
        # print(hash(person))
        images_ = []
        # images_nominal_indices_ = []
        prev_index = []


        with open(os.path.join(base_path, "vgg_face_dataset", "files", person), 'r') as f:
            lines = f.readlines()       
        

        while len(images_) < images_size[person]:
            index = random.randrange(0, 1000)
            logging.debug("Index = " + str(index))
            if index in prev_index:
                logging.debug("Index = " + str(index) + " => already there")
                continue
            else:
                prev_index.append(index)
                line = lines[index]
                # only curated data
                if int(line.split(" ")[8]) == 1:
                    url = line[line.find("http://"): line.find(".jpg") + 4]
                    logging.debug("URL > \"" + str(url))
                    try:
                        res = request.urlopen(url, timeout = 1)
                        img = np.asarray(bytearray(res.read()), dtype="uint8")
                        img = cv2.imdecode(img, cv2.IMREAD_COLOR)

                        h, w = img.shape[:2]
                        cv2_imshow(cv2.resize(img, (w//4, h//4)))
                        # images_nominal_indices_.append(index)

                        filename = path_database +  str(index) + "_" + str(person.split(".")[0]) + ".jpg"

                        value = cv2.imwrite(filename, img) 
                        # logging.debug("saved in DB: " + str(filename))
                        images_.append(img)
                        fo.write(line)
                    except ValueError as e:
                            logging.error("Value Error >" + str(e))
                    except (HTTPError, URLError) as e:
                            logging.error('ERROR RETRIEVING URL >' + str(e))
                    except timeout:
                            logging.error('socket timed out - URL %s', str(url))
                    except cv2.error as e: 
                            logging.error("ERROR WRITING FILE IN DB  >" + str(e))
                    except:
                        logging.error("Weird exception : " + str(line))
                else:
                    logging.debug("File not curated => rejected (id = " + str(index) + " )")    
                
                # images[person] = images_
                # images_nominal_indices[person] = images_nominal_indices_

    fo.close()
else:
    logging.warning("If you really want to erase and renew the database, please change first the \"confirmation\" boolean variable, at the beginning of this cell")



From local database (not the web, not the drive), building lists of images for all persons

In [0]:
with open(file_info, 'r') as f: 
    lines = f.readlines()

assert len(lines)==total_images_size, "amount of lines in file incompatible" 

images = {}

for p in persons:
    images[p] = []


images_index = {}
for running_index in range(len(lines)):
    if running_index in range(0,images_size[personA]):
        p = personA
    elif running_index in range(images_size[personA],images_size[personA]+images_size[personB]):
        p = personB
    elif running_index in range(images_size[personA]+images_size[personB],images_size[personA]+images_size[personB]+images_size[personC]):
        p = personC
    elif running_index in range(images_size[personA]+images_size[personB]+images_size[personC],total_images_size):
        p = personD
    ind = str(int(lines[running_index].split(" ")[0])-1)
    vgg_ids[p].append(ind)
    filename = ind + "_" + str(p.split(".")[0]) + ".jpg"
    images[p].append(cv2.imread(path_database+filename, cv2.IMREAD_COLOR))
    
# print(len(images.keys()))
# print(len(images[personA]))
# print(len(images[personB]))
# print(len(images[personC]))
# print(len(images[personD]))

# for person in persons:
#     counter = 0
#     for img in images[person]:
#         logging.debug("------------------------------------------------------")
#         logging.debug("Photo ID = " + str(counter))
#         cv2_imshow(img)
#         counter += 1

To remove:
From DB loaded from the internet, several images are to be removed. The main reasons are:


*   Too much make up
*   too different hair with usual representation
*   really poor image quality
*   relevance and error in dataset
*   same image as already in dataset
*   cropped image

considering the tight selection of images to train our model, and the relative global amount of image candidates, it is acceptable to first sort the images according to visual insights. 

From the initial retrieved images, we then remove the undesired images, that we copy in discard images folder, for tracking purposes. We may want to use them later on to assess the training, for an academical purpose.

From the remaing imaging, we can apply the same strategy of selecting the required datasets, that we finally load in separate folders, and save in drive.



In [0]:
# Dictionary of the size required (see section 3)
datasets_size = {}
datasets_size[personA] = 30
datasets_size[personB] = 30
datasets_size[personC] = 10
datasets_size[personD] = 10


# manually remove images that are not relevant or considered not good enough to be part of the dataset
to_remove = {}
to_remove[personA] = [0,1,4,8,12,13,16,23,28,34,36,42,44,47,48,49,54]
to_remove[personB] = [4,7,11,12,13,16,21,22,23,25,26,27,32,36,39,41,46,49,53,55,58]
to_remove[personC] = [0,1,4,6,7,11,14,16,17,19,20]
to_remove[personD] = [0,3,5,6,8,10,15,16,17,24]

# goal is to sort in descending to remove elements from lists without modifying the indexes
for p in persons:
    to_remove[p].sort(reverse = True)


# retrieve images candidates
# --------------------------
if len(os.listdir(path_datasets) ) == 0 or True:
    logging.debug("datasets empty - need to retrieve all !")
    # removing images to discard
    for person in persons:
        for index in to_remove[person]:
            img = images[person].pop(index)
            logging.debug("Removing item " + str(index) + " from list " + str(person))
            try:
                filename = path_discard +  str(index) +"_discarded_" + str(person.split(".")[0]) + ".jpg"
                cv2.imwrite(filename, img) 
            except:
                logging.error("Error while writing discarded image " + str(filename))

    # randomly select among remaining images
    for person in persons:
        # build list of indices from remaining images
        logging.debug("Phase 2 -> random selection for " + str(person))

        images_ = []
        indices = []
        new_ids = []
        # prev_index = []
        random.seed(person)

        while len(indices) < datasets_size[person]:       
            index = random.randrange(0, len(images[person]))
            if index in indices:
                logging.debug("Index among remaining = " + str(index) + " => already there")
                continue
            else:
                # prev_index.append(index)
                indices.append(index)

        logging.debug("Phase 2 -> random selection idx:  " + str(indices))

        for index in indices:
            img = images[person][index]
            images_.append(img)
            filename = path_datasets +  str(vgg_ids[person][index]) + "_" + str(person.split(".")[0]) + ".jpg"
            logging.debug("saved: " + str(filename))
            cv2.imwrite(filename, img) 
            new_ids.append(vgg_ids[person][index])
        images[person] = images_
        vgg_ids[person] = new_ids
else:
    logging.debug("folders not empty => can build directly images dictionnary")

# logging.debug("Number of images keys=" + len(images.keys))
# logging.debug("Number of images values=" + len(images.values))

logging.info(pretty_return_dict_size(images))

In [0]:
# Save to drive folders
to_db_confirmation = False

path_drive_DB = r"/content/drive/My Drive/ComputerVision/DATABASE"
path_drive_Datasets = r"/content/drive/My Drive/ComputerVision/DATASETS"



# drive folders should be properly set up

if to_db_confirmation:
    try:
        drive.mount('/content/drive')
    except:
        pass
    try:
        shutil.rmtree(path_drive_DB)
        shutil.rmtree(path_drive_Datasets)
    except:
        logging.error("Error in rmtree") 


    try: 
        os.mkdir(path_drive_DB) 
        os.mkdir(path_drive_Datasets)
    except OSError as error: 
        logging.error(error) 
    
    logging.debug("Saving database in drive : start")

    fromDirectory = path_database
    toDirectory = path_drive_DB
    copy_tree(fromDirectory, toDirectory)

    logging.debug("Saving datasets in drive : start")

    fromDirectory = path_datasets
    toDirectory = path_drive_Datasets
    copy_tree(fromDirectory, toDirectory)

    logging.debug("Saving: done !")


<!-- ## New Section title
New section texte.

[square]
- List item1
** * * *   List item2
*   List item3
    * list item 31
    ** list item 32 -->






In [0]:
def draw_box(information_file_lines, index, image, person):
    """
    information_file_lines is a list of lines of all images in the database
    index is the vgg_idx
    image is the image of interest, as retrieved from website
    """
    raise RuntimeError("Probably not the smartest way...")
    logging.debug("VGG INDEX = " + str(index))

    if person == personA:
        lines = information_file_lines[0:59]
    elif person == personB:
        lines = information_file_lines[59:119]
    elif person == personC:
        lines = information_file_lines[119:149]
    elif person == personD:
        lines = information_file_lines[150:179]
    
    for line in lines:
        if int(line.split(" ")[0]) == index:
            h, w = img_.shape[:2]
            
            left = int(round(float(line.split(" ")[2])))
            # left = w - left
            top = int(round(float(line.split(" ")[3])))
            # top = h - top
            right = int(round(float(line.split(" ")[4])))
            # right = w - right
            bottom = int(round(float(line.split(" ")[5])))
            # bottom = h - bottom
            image = cv2.rectangle(image, (left,top), (right,bottom), [0,0,255], 10)

        else:
            continue

    return image

recognize faces based on haarCascade

tutorial in: [How to detect faces using Haar Cascade](https://www.digitalocean.com/community/tutorials/how-to-detect-and-extract-faces-from-an-image-with-opencv-and-python)


In [0]:
faceCascade = cv2.CascadeClassifier(os.path.join(base_path, "haarcascade_frontalface_default.xml"))
faces_cropped = {}

with open(file_info, 'r') as f: 
    lines = f.readlines()

for person in persons:

    faces_cropped[person] = []

    for img in images[person]:
        img_ = img.copy()
        img_gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(
            img_gray,
            scaleFactor=1.13,
            minNeighbors=10,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        for (x,y,w,h) in faces:
            faces_cropped[person].append(img[y:y+h, x:x+w])
            cv2.rectangle(img_, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # h, w = img_.shape[:2]
        # draw_box(lines, int(vgg_ids[person][running_index])+1, img_, person)
        # cv2_imshow(cv2.resize(img_, (w // 5, h // 5)))
logging.info("Faces extracted and saved in dictionnary")
logging.info(pretty_return_dict_size(faces_cropped))

From Datasets of images to Trainng set and validation sets;

```
images[person]
```
is a dictionary of all the images in the database that are dedicated to a specific person. In order to obtain a :
* training set
* test set

for the person A and B, one can randomly select 20 images for validation sets, and 10 images for test sets. 

For the sake of reproducibility, the details are logged

In [0]:
training_sets_size = {}
training_sets_size[personA] = 20
training_sets_size[personB] = 20
training_sets_size[personC] = 0
training_sets_size[personD] = 0

test_sets_size = {}
test_sets_size[personA] = 10
test_sets_size[personB] = 10
test_sets_size[personC] = 10
test_sets_size[personD] = 10

training_set = {}
test_set = {}
for person in persons:
    image_ = faces_cropped[person]
    training_set_ = []
    random.seed(person)
    init_set = set(range(0, len(image_)))

    indices_training = random.sample(init_set, training_sets_size[person])
    indices_test = list(init_set - set(indices_training))

    training_set[person] = [faces_cropped[person][i] for i in indices_training] 
    test_set[person] = [faces_cropped[person][i] for i in indices_test]



In [0]:
# show_images_from_dict(training_set)

In [0]:
# show_images_from_dict(test_set)


***AJOUTER UNE VISUALISATION*** 

In [0]:
#  Visualisation des images prises

logging.error("NO visualisation found !")

Feature Descriptor - HOG

In [0]:
src = faces_cropped[personA][0]
cv2_imshow(src)

#1 resizing
resized_img = cv2.resize(src, (128,128))

fd, hog_image = hog(resized_img, 
                    orientations=9, 
                    pixels_per_cell=(8,8), 
                    cells_per_block=(2, 2), 
                    block_norm = "L2",
                    visualize=True, 
                    transform_sqrt = True,
                    multichannel=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), sharex=True, sharey=True) 

ax1.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Input image') 

# Rescale histogram for better display 
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 

ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients - rescaled')
print("HOG Rescaled: " + str(hog_image_rescaled.min()) + " -> " + str(hog_image_rescaled.max()) )

# ax3.imshow(hog_image, cmap=plt.cm.gray) 
# ax3.set_title('Histogram of Oriented Gradients')
# print("HOG: " + str(hog_image.min()) + " -> " + str(hog_image.max()) )
plt.show()

print(hog_image_rescaled.shape)
print(fd.shape)

## Detail procedure for HOG
# computation of the gradients
* each pixel
* magnitude and orientation
    * unsigned gradient : On the right, we see the raw numbers representing the gradients in the 8×8 cells with one minor difference — the angles are between 0 and 180 degrees instead of 0 to 360 degrees. These are called “unsigned” gradients because a gradient and it’s negative are represented by the same numbers. In other words, a gradient arrow and the one 180 degrees opposite to it are considered the same. But, why not use the 0 – 360 degrees ? Empirically it has been shown that unsigned gradients work better than signed gradients for pedestrian detection. Some implementations of HOG will allow you to specify if you want to use signed gradients
    
* plot on a cell
* size of the cell
    * design choice


In [0]:

img = faces_cropped[personA][0].copy()
img = cv2.imread("hog-preprocessing.jpg")
logging.debug("Toy Example")
cv2_imshow(img)

y=95
x=200
h = 172
w = 84
crop_img = img[y:y+h, x:x+w]
crop_img = cv2.resize(crop_img, (64,128))
cv2_imshow(crop_img)

img = np.float64(crop_img)

gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = 1)
gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = 1)

mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
orn = angle.copy()

# Normatlization to have a magnitude between 0 and 255
mag_normalized = np.array(((mag - mag.min()) / (mag.max() - mag.min()) * 255 + 0)).astype(np.uint8)
# orn_normalized = np.array(((orn - orn.min()) / (orn.max() - orn.min()) * 360 + 0)).astype(np.float64)
orn_normalized = orn

# mag_normalized = np.array(((mag) / (mag.max() - mag.min()) * 255 + 0)).astype(np.uint8)
# orn_normalized = np.array(((orn) / (orn.max() - orn.min()) * 180 + 0)).astype(np.uint8)

# mag_normalized = ((mag) * 255).astype(np.uint8)
# orn_normalized = orn 

# mag_normalized = mag_normalized / np.sqrt(np.sum(mag_normalized**2))
# orn_normalized = orn_normalized / np.sqrt(np.sum(orn_normalized**2))

k = 8
y_ = 10
x_ = 24
crop_img_cell = crop_img[y_:y_+k, x_:x_+k]



fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 8), sharex=True, sharey=True) 

ax1.imshow(cv2.cvtColor(crop_img_cell, cv2.COLOR_BGR2RGB))
ax1.set_title('Input image') 


# construction of the magnitude and orn matrices
orn_matrix = np.zeros((k,k))
for i in range(k):
    for j in range(k):
        idx = np.argmax(mag_normalized[y_+j,x_+i])
        angle_ = round(orn[y_+j,x_+i, idx])
        orn_matrix[i,j] = angle_

orn_matrix_clipped = orn_matrix.copy() 

orn_matrix_clipped = ((orn_matrix_clipped)+ 90 )%360
for i in range(k):
    for j in range(k):
        if 0 <= orn_matrix_clipped[i,j] < 180:
            orn_matrix_clipped[i,j] = 180 - orn_matrix_clipped[i,j]
        elif 180 <= orn_matrix_clipped[i,j] <=359.9:
            orn_matrix_clipped[i,j] = 180 - orn_matrix_clipped[i,j] % 180

orn_matrix_clipped.astype(np.uint8)
# ax1 => just to visually represent the arrows
for i in range(k):
    for j in range(k):
        r = round(mag_normalized[y_+j,x_+i].max())/(mag_normalized.max() - mag_normalized.min())
        angle_ = orn_matrix[i,j]
        ax1.arrow(i, j, r*np.cos(np.deg2rad(angle_)), r*np.sin(np.deg2rad(angle_)))

for i in range(k):
    for j in range(k):
        c = orn_matrix_clipped[i,j]
        ax2.text(i, j, str(c), fontsize=10,va='center', ha='center')

ax2.matshow(orn_matrix_clipped, alpha=0)
ax2.set_title('Orn values')


# ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax3.set_title('Magnitude values')
# print("HOG Rescaled: " + str(hog_image_rescaled.min()) + " -> " + str(hog_image_rescaled.max()) )

intersection_matrix = np.zeros(crop_img_cell.shape)

for i in range(k):
    for j in range(k):
        c = round(mag_normalized[y_+j,x_+i].max())
        ax3.text(i, j, str(c), fontsize=10,va='center', ha='center')
        intersection_matrix[j,i] = c
ax3.matshow(intersection_matrix)
# ax2.imshow(cv2.cvtColor(crop_img_cell, cv2.COLOR_BGR2RGB))

plt.show()


histogram construction is based on the gradient computed - both magnitude and orientation (as defined)

- the bin is selected according to the orientation (direction) of the gradient; 
- the value that goes in the bin is based on the magnitude

For instance on the toy example:
first pixel has:
* mag = 6; orn = 45°. So, the vote of this pixel goes for 75% in the bin of 40°, and 25% in the bin of 60°, as closer to 40°. As a result, we add 4.5 to bin nb 3, and 1.5 to bin nb 4. 

Computations of the bins - pile up on our cell

In [0]:
N_BUCKETS = 9
bin_list = np.zeros(N_BUCKETS)

def fill_bins_one_pixel(mag, orn, bin_list):
    size_bin = 20.
    if orn >= 160:
        left_bin = 8
        right_bin = 9
        left_val= mag * (right_bin * 20 - orn) / 20
        right_val = mag * (orn - left_bin * 20) / 20
        left_bin = 8
        right_bin = 0
    else:
        left_bin = int(orn / size_bin)
        right_bin = (int(orn / size_bin) + 1) % N_BUCKETS
        left_val= mag * (right_bin * 20 - orn) / 20
        right_val = mag * (orn - left_bin * 20) / 20

    bin_list[left_bin] += left_val
    bin_list[right_bin] += right_val

print(bin_list)

toy_dir = np.array([[80,36,5,10,0,64,90,73],
                    [37,9,9,179,78,27,169,166],
                    [87,136,173,39,102,163,152,176],
                    [76,13,1,168,159,22,125,143],
                    [120,70,14,150,145,144,145,143],
                    [58,86,119,98,100,101,133,113],
                    [30,65,157,75,78,165,145,124],
                    [11,170,91,4,110,17,133,110]])
toy_mag = np.array([[2,3,4,4,3,4,2,2],
                    [5,11,17,13,7,9,3,4],
                    [11,21,23,27,22,17,4,6],
                    [23,99,165,135,85,32,26,2],
                    [91,155,133,136,144,152,57,28],
                    [98,196,76,38,26,60,170,51],
                    [165,60,60,27,77,85,43,136],
                    [71,13,34,23,108,27,48,110]])
print(mag_normalized.shape)

for i in range(8):
    for j in range(8):
        m = mag_normalized[i,j].max()
        d = orn_matrix_clipped[i,j]
        fill_bins_one_pixel(m,d,bin_list)

print(bin_list)

n = np.linalg.norm(bin_list)
bin_norms = bin_list/n
plt.bar( range(9), bin_norms)
plt.show()
print(bin_norms)

fd, hog_image = hog(crop_img_2, 
                    orientations=9, 
                    pixels_per_cell=(8,8), 
                    cells_per_block=(1, 1), 
                    block_norm = "L2",
                    visualize=True, 
                    transform_sqrt = False,
                    multichannel=True)
print(fd)
plt.bar( range(9), fd)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 8), sharex=True, sharey=True) 

ax1.imshow(cv2.cvtColor(crop_img_cell, cv2.COLOR_BGR2RGB))
ax1.set_title('Input image') 

# Rescale histogram for better display 
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 

ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients - rescaled')

print(hog_image_rescaled)
plt.show()


Gradient is sensitive to light variation. We do not want that => normalization the histogram so that they are not affected by lighting variations

> $v = [x, y, z]$, 

> $\|v\| = \sqrt{x^2 + y^2 + z^2}$

> $v_{normalized} =  \frac{v}{\|v\|} = \Big[ \frac{x}{\|v\|}, \frac{y}{\|v\|}, \frac{z}{\|v\|} \Big]$

The normalization has a stronger helpful effect if applied on a larger scale than the cell only. AS a result, an often good choice is to apply on 4 cells, or a 16x16 block.  Four histograms are taken into account. 
* one cell (8 x 8) histogram is a vector of size 9 (as 9 bins)
* one block (16 x 16) histogram is the concatenation of the 4 histograms, each representing one cell of the block, hence represented by a (36 x 1) vector.
* the final HOG feature vector is based on the concatenation of all blocks.
If the image as a width of size w*8 pixels, and a height of h*8 pixels, 
the image dimension in (8*h x 8*w). In such an image, they are :
    -   h cells vertically, and w cells horizontally,
    -  (h-1) blocks vertically and (w-1) blocks horizontally.

For an image of 128 x 128, we then have:
* w = 128/8 = 16
* h = 128/8 = 16
* #cells = 16x16
* #Blocks = 15 * 15 = 225 blocks

each block having a representative vector (36x1), the resulting vector has dimension (225*36 x 1), or (8100x1)
